Preparação e Limpeza de Dados

In [44]:
# Importação das Bibliotecas
import pandas as pd
import numpy as np

In [45]:
# Carregamento e leitura das bibliotecas
df = pd.read_csv("docs/treino.csv")

df.head()


,op_padronizado,dt_ocorrencia,hr_ocorrencia,uf,regiao,latitude,longitude,cat_aeronave,fase_operacao,modelo_aeronave,peso_max_decolagem,numero_assentos,nome_fabricante,les_fatais_trip
0,AEROAGRICOLA CHAPADAO LTDA,21/12/1999,NaN,SP,Sudeste,NaN,NaN,S05,NaN,PA-25-235,1315.0,1.0,PIPER AIRCRAFT,0
1,RONDON AVIACAO AGRICOLA LTDA,06/01/2000,NaN,MT,Centro-Oeste,NaN,NaN,S05,NaN,EMB-201,1800.0,1.0,NEIVA,0
2,SAPA SERV. AEREO DE PROTECAO AGRICOLA,22/02/2000,NaN,RS,Sul,NaN,NaN,S05,NaN,A188B,1497.0,1.0,CESSNA AIRCRAFT,0
3,AERO AGRICOLA CAICARA LTDA,10/05/2000,NaN,SP,Sudeste,NaN,NaN,S05,NaN,PA-25-260,1315.0,1.0,PIPER AIRCRAFT,0
4,AVIACAO AGRICOLA RECREIO LTDA,09/11/2000,NaN,Indeterminado,NaN,NaN,NaN,TPP,NaN,369HS,1157.0,5.0,HUGHES HELICOPTER,0


 Tratamento de dados

In [35]:
# Pegar informações da Tabela
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   op_padronizado      534 non-null    object 
 1   dt_ocorrencia       536 non-null    object 
 2   hr_ocorrencia       365 non-null    object 
 3   uf                  536 non-null    object 
 4   regiao              473 non-null    object 
 5   latitude            382 non-null    object 
 6   longitude           382 non-null    object 
 7   cat_aeronave        536 non-null    object 
 8   fase_operacao       393 non-null    object 
 9   modelo_aeronave     533 non-null    object 
 10  peso_max_decolagem  534 non-null    float64
 11  numero_assentos     534 non-null    float64
 12  nome_fabricante     533 non-null    object 
 13  les_fatais_trip     536 non-null    int64  
dtypes: float64(2), int64(1), object(11)
memory usage: 58.8+ KB


In [47]:
# Bloco 2: Inspeção inicial e remoção de duplicatas

print("--- Informações Iniciais do DataFrame ---")
df.info()

# Verifica e remove linhas duplicadas
linhas_antes = len(df)
df = df.drop_duplicates().reset_index(drop=True)
print(f"Linhas antes da remoção de duplicatas: {linhas_antes}")
print(f"Linhas após a remoção de duplicatas: {len(df)}")

--- Informações Iniciais do DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   op_padronizado      508 non-null    object 
 1   dt_ocorrencia       510 non-null    object 
 2   hr_ocorrencia       340 non-null    object 
 3   uf                  510 non-null    object 
 4   regiao              447 non-null    object 
 5   latitude            358 non-null    object 
 6   longitude           358 non-null    object 
 7   cat_aeronave        510 non-null    object 
 8   fase_operacao       367 non-null    object 
 9   modelo_aeronave     507 non-null    object 
 10  peso_max_decolagem  508 non-null    float64
 11  numero_assentos     508 non-null    float64
 12  nome_fabricante     507 non-null    object 
 13  les_fatais_trip     510 non-null    int64  
dtypes: float64(2), int64(1), object(11)
memory usage: 55.9+ KB
Linha

In [ ]:
# Converte latitude e longitude para números, trocando a vírgula por ponto
print("Convertendo 'latitude' e 'longitude' para Float")
df['latitude'] = df['latitude'].astype(str).str.replace(',', '.').astype(float)
df['longitude'] = df['longitude'].astype(str).str.replace(',', '.').astype(float)

# Converte a coluna de data para o formato datetime
# O interpretado não consegue entender o formato Object para data e hora
print("Convertendo 'dt_ocorrencia' para formato de data...")
df['dt_ocorrencia'] = pd.to_datetime(df['dt_ocorrencia'], format='%d/%m/%Y', errors='coerce')

# Verifica o tipo de dados após a conversão
df.info()

Convertendo 'latitude' e 'longitude' para Float
Convertendo 'dt_ocorrencia' para formato de data...

--- Tipos de dados após a conversão ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   op_padronizado      508 non-null    object        
 1   dt_ocorrencia       510 non-null    datetime64[ns]
 2   hr_ocorrencia       340 non-null    object        
 3   uf                  510 non-null    object        
 4   regiao              447 non-null    object        
 5   latitude            358 non-null    float64       
 6   longitude           358 non-null    float64       
 7   cat_aeronave        510 non-null    object        
 8   fase_operacao       367 non-null    object        
 9   modelo_aeronave     507 non-null    object        
 10  peso_max_decolagem  508 non-null    float64       
 11  numero_assentos     5

In [38]:
# Comando para verificar se há valores nulos na tabela
print(df.isnull().sum())

op_padronizado        0
dt_ocorrencia         0
hr_ocorrencia         0
uf                    0
regiao                0
latitude              0
longitude             0
cat_aeronave          0
fase_operacao         0
modelo_aeronave       0
peso_max_decolagem    0
numero_assentos       0
nome_fabricante       0
les_fatais_trip       0
ano_ocorrencia        0
mes_ocorrencia        0
dtype: int64


In [39]:
# A biblioteca LabelEncoder do scikit-learn serve para transformar dados (textos ou rótulos) em números inteiros
from sklearn.preprocessing import LabelEncoder

# Lista das colunas categóricas que você quer transformar
colunas_categoricas = ['fase_operacao', 'cat_aeronave', 'regiao', 'uf', 'modelo_aeronave', 'nome_fabricante']

# Aplica LabelEncoder em cada coluna da lista
for col in colunas_categoricas:
    df[col] = LabelEncoder().fit_transform(df[col])


C:\Users\Vitor\AppData\Local\Temp\ipykernel_6632\3080248961.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\Vitor\AppData\Local\Temp\ipykernel_6632\3080248961.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\Vitor\AppData\Local\Temp\ipykernel_6632\3080248961.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [40]:
# Escolha as colunas que fazem sentido para prever acidentes
X = df[['latitude', 'longitude', 'peso_max_decolagem', 'numero_assentos', 'fase_operacao', 'cat_aeronave', 'regiao', 'uf']]
y = df['les_fatais_trip']

# X vai ser as colunas que queremover verificar no Y (Se teve acidentes fatais ou não)
# Então adiciona tudo as tabelas por enquanto, mas é adaptável conforme formos estudando

In [41]:

# Essa bibliteca importa uma função do scikit-learn na qual faz dois conjuntos, um treina o conjunto e outro testa se está correto
# Isso evita que o modelo não "decore" os dados, e sim aprenda
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [42]:
# Serve para criar uma arvore de decisão para classificação dos dados
from sklearn.tree import DecisionTreeClassifier

# Treina o modelo com base no X_train, Y_train e verifica as respostas corretas com  base no Y_test, Y_test
model = DecisionTreeClassifier()
model.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [43]:
# Importa funções que calculam métricas de avaliações
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Acurácia: 0.6944444444444444
Precisão: 0.125
Recall: 0.2
F1 Score: 0.15384615384615385
